In [1]:
import numpy as np
import torch
import joblib
from scipy.spatial.distance import cdist


In [2]:
eval_labels = []

for i in range(10):
  t = torch.load('./dataset/part_one_dataset/eval_data/' + str(i+1) + '_eval_data.tar.pth')
  eval_labels.append(t['targets'])

for i in range(10):
    t = torch.load('./dataset/part_two_dataset/eval_data/' + str(i+1) + '_eval_data.tar.pth')
    eval_labels.append(t['targets'])

/var/folders/15/rhh2dgys0n75gk26z6nrx13c0000gn/T/ipykernel_18365/4124738520.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load('./dataset/part_one_dataset/ev

In [3]:
eval_data_extracted_1 = np.load('./features/Part-1/eval_data_extracted.npy', allow_pickle=True)
eval_data_extracted_2 = np.load('./features/Part-2/eval_data_extracted.npy', allow_pickle=True)

In [4]:
eval_data = np.concatenate((eval_data_extracted_1, eval_data_extracted_2), axis=0)

In [5]:
class LwPClassifier:
    def __init__(self, metric='euclidean'):
        self.metric = metric
        self.class_prototypes = {}

    def fit(self, X, y, weights=None, alpha=0.01):
        """Compute prototypes (mean vectors) for each class, with optional weighting and regularization."""
        if len(X.shape) > 2:
            X = X.reshape(X.shape[0], -1)
        
        self.classes_ = np.unique(y)
        if weights is None:
            weights = np.ones_like(y, dtype=float)

        self.class_prototypes = {
            cls: np.average(X[y == cls], axis=0, weights=weights[y == cls]) - alpha * np.mean(X, axis=0) 
            for cls in self.classes_
        }

    def predict(self, X):
        """Predict class for each sample in X."""
        if len(X.shape) > 2:
            X = X.reshape(X.shape[0], -1)
        prototypes = np.array([self.class_prototypes[cls] for cls in self.classes_])
        distances = cdist(X, prototypes, metric=self.metric)
        closest_prototype_idx = distances.argmin(axis=1)
        return self.classes_[closest_prototype_idx]

    def retrain(self, X, y, momentum=0.9):
        """Update prototypes with new data using adaptive learning and momentum."""
        if len(X.shape) > 2:
            X = X.reshape(X.shape[0], -1)
        
        for cls in np.unique(y):
            if cls in self.class_prototypes:
                old_prototype = self.class_prototypes[cls]
                new_prototype = X[y == cls].mean(axis=0)
                self.class_prototypes[cls] = momentum * old_prototype + (1 - momentum) * new_prototype
            else:
                self.class_prototypes[cls] = X[y == cls].mean(axis=0)
    
    def retrain_with_unlabeled_data(self, X, threshold=0.8):
        """Adapt model to unlabeled data by generating pseudo-labels based on current prototypes."""
        if len(X.shape) > 2:
            X = X.reshape(X.shape[0], -1)
        
        distances = cdist(X, np.array(list(self.class_prototypes.values())), metric=self.metric)
        predicted_labels = np.argmin(distances, axis=1)
        predicted_confidences = np.min(distances, axis=1)
        
        confident_indices = predicted_confidences < threshold
        pseudo_labels = predicted_labels[confident_indices]
        X_confident = X[confident_indices]
        
        self.retrain(X_confident, pseudo_labels)

    def score(self, X, y):
        """Calculate accuracy of the model."""
        predictions = self.predict(X)
        return np.mean(predictions == y)


In [6]:
f1 = joblib.load('./models/Part-1/f1.joblib')
f2 = joblib.load('./models/Part-1/f2.joblib')
f3 = joblib.load('./models/Part-1/f3.joblib')
f4 = joblib.load('./models/Part-1/f4.joblib')
f5 = joblib.load('./models/Part-1/f5.joblib')
f6 = joblib.load('./models/Part-1/f6.joblib')
f7 = joblib.load('./models/Part-1/f7.joblib')
f8 = joblib.load('./models/Part-1/f8.joblib')
f9 = joblib.load('./models/Part-1/f9.joblib')
f10 = joblib.load('./models/Part-1/f10.joblib')


In [7]:
f11 = joblib.load('./models/Part-2/f11.joblib')
f12 = joblib.load('./models/Part-2/f12.joblib')
f13 = joblib.load('./models/Part-2/f13.joblib')
f14 = joblib.load('./models/Part-2/f14.joblib')
f15 = joblib.load('./models/Part-2/f15.joblib')
f16 = joblib.load('./models/Part-2/f16.joblib')
f17 = joblib.load('./models/Part-2/f17.joblib') 
f18 = joblib.load('./models/Part-2/f18.joblib')
f19 = joblib.load('./models/Part-2/f19.joblib')
f20 = joblib.load('./models/Part-2/f20.joblib')


# Testing accuracies

In [ ]:
models = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20]

In [16]:
accuracies_part1 = [[] for _ in range(10)]

for i in range(1, 10+1):
    for j in range(1, i+1):
        model_id = i-1
        data_id = j-1
        score = models[model_id].score(eval_data[data_id], eval_labels[data_id])
        accuracies_part1[model_id].append(score)

print("Task 1")
for i in range(10):
    print(accuracies_part1[i])

Task 1
[0.8716]
[0.8712, 0.872]
[0.8688, 0.87, 0.862]
[0.8692, 0.8676, 0.8596, 0.868]
[0.8664, 0.8636, 0.8592, 0.8684, 0.8652]
[0.8648, 0.8624, 0.8592, 0.8668, 0.8648, 0.8664]
[0.8644, 0.8616, 0.8572, 0.866, 0.864, 0.864, 0.8564]
[0.864, 0.8604, 0.856, 0.8648, 0.8632, 0.8632, 0.8564, 0.8608]
[0.8628, 0.8592, 0.856, 0.8632, 0.864, 0.8612, 0.856, 0.86, 0.8488]
[0.8604, 0.8588, 0.8564, 0.8624, 0.8632, 0.8604, 0.8544, 0.86, 0.8464, 0.8568]


In [19]:
accuracies_part2 = [[] for _ in range(10)]
for i in range(1, 10+1):
    for j in range(1, 10+i+1):
        model_id = i-1+10
        data_id = j-1
        score = models[model_id].score(eval_data[data_id], eval_labels[data_id])
        accuracies_part2[model_id-10].append(score)

print("Task 2")
for i in range(10):
    print(accuracies_part2[i])

Task 2
[0.858, 0.8544, 0.85, 0.8564, 0.8604, 0.8572, 0.8508, 0.8576, 0.838, 0.8544, 0.7028]
[0.8548, 0.852, 0.8448, 0.8524, 0.8536, 0.852, 0.8456, 0.8536, 0.838, 0.8512, 0.7004, 0.5244]
[0.8508, 0.8484, 0.8432, 0.8484, 0.8524, 0.8468, 0.8416, 0.85, 0.8344, 0.8472, 0.6972, 0.5256, 0.7376]
[0.8476, 0.846, 0.8408, 0.8468, 0.8532, 0.8464, 0.84, 0.8484, 0.8324, 0.8452, 0.6932, 0.5204, 0.736, 0.806]
[0.8476, 0.8456, 0.8412, 0.8476, 0.8532, 0.8456, 0.8388, 0.848, 0.8316, 0.8452, 0.692, 0.5188, 0.7324, 0.8044, 0.832]
[0.8412, 0.8436, 0.8388, 0.8464, 0.8524, 0.8436, 0.8388, 0.8464, 0.8296, 0.842, 0.688, 0.5192, 0.7336, 0.7992, 0.8316, 0.7144]
[0.8424, 0.8408, 0.8368, 0.8452, 0.8496, 0.842, 0.8372, 0.8456, 0.8268, 0.838, 0.6908, 0.5152, 0.7352, 0.798, 0.8284, 0.7128, 0.776]
[0.8388, 0.8396, 0.8312, 0.8392, 0.8452, 0.8392, 0.8316, 0.8404, 0.8244, 0.8364, 0.6848, 0.5148, 0.7324, 0.7968, 0.8232, 0.7096, 0.772, 0.7072]
[0.8276, 0.8316, 0.8292, 0.8324, 0.8412, 0.832, 0.824, 0.8336, 0.8204, 0.8316, 0.